In [ ]:
from IPython import get_ipython

ipython = get_ipython()

exec_no = ipython.execution_count
exec_no

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random 
import pickle
import os

from IPython.utils import io

from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.cluster import DBSCAN, KMeans, AgglomerativeClustering
from sklearn.manifold import TSNE
from sklearn.base import clone
from sklearn.ensemble import RandomForestClassifier

#!pip install -U git+https://github.com/joaopfonseca/SOMPY.git
import sompy
from sompy.visualization.mapview import View2D
from sompy.visualization.bmuhits import BmuHitsView
from sompy.visualization.hitmap import HitMapView

%pip install boruta
from boruta import BorutaPy


%matplotlib inline
%config InlineBackend.figure_format = 'retina' # optionally, you can change 'svg' to 'retina'


In [ ]:
if False: 
    with io.capture_output() as captured:
        %run get_wd.py

    s = captured.stdout # prints stdout from your script

In [ ]:
if exec_no == 1: 
    s = os.getcwd()
    os.chdir(os.path.dirname(s))
    exec_no += 1
print(os.getcwd())

In [ ]:
# definitions 

#os.chdir('/Users/dp/Nova/OneDrive - NOVAIMS/1stSemester/DM/DMProject')
computed_data_path = 'computed_data/'
explorations_data_path = 'explorations/'

paths = [computed_data_path, explorations_data_path]
for path in paths:
    if not os.path.exists(path): 
        os.makedirs(path)

In [ ]:
#run promo_history.ipynb


In [ ]:
# save results

with open(os.path.join(computed_data_path, 'history_feat_raw.pickle'), 'rb') as f: 
    history_agg = pickle.load(f)
    
with open(os.path.join(computed_data_path, 'history_feat_multi_out.pickle'), 'rb') as f: 
    history_agg_out_multi = pickle.load(f)
    
with open(os.path.join(computed_data_path, 'history_feat_multi_clean.pickle'), 'rb') as f: 
    history_agg_multi = pickle.load(f)

In [ ]:
history_agg_multi

In [ ]:
history_agg_out_multi

# Preparation

## setup

In [ ]:
# define cols

## cols for clsutering 
cl_feat_hist = history_agg_multi.columns.to_list()
print('\ncl_feat_hist: \n',cl_feat_hist)

metric_features = cl_feat_hist

## cols for descirption 
#desc_feat_hist = history_agg.loc[~history_agg.columns.isin(cl_feat_hist),:].columns.to_list()
desc_feat_hist = history_agg.columns[~history_agg.columns.isin(cl_feat_hist)].to_list()
print('\ndesc_feat_hist: \n',desc_feat_hist)



In [ ]:
# define rows

cl_donor_ids = history_agg_multi.index.to_list()


In [ ]:
history_agg[history_agg.index.isin(cl_donor_ids)]

In [ ]:
df_hist = history_agg_multi[cl_feat_hist]
df_hist


## Scale

In [ ]:
def scale_df(df, scale='minmax'): 
    if scale == 'minmax': 
        scaler = MinMaxScaler()
    df_normal = scaler.fit_transform(df)
    df_normal = pd.DataFrame(df_normal, columns=df.columns, index=df.index)
    return df_normal

    
df_hist_normal = scale_df(df=df_hist, scale='minmax')

## merge df

In [ ]:
df_normal = df_hist_normal.copy()

## multivariate outliers

# Clustering

In [ ]:
cl_names = set()



In [ ]:
# Applying the right clustering (algorithm and number of clusters) for each perspective
kmeans_prod = KMeans(
    n_clusters=3,
    init='k-means++',
    n_init=20,
    random_state=1
)
hist_labels = kmeans_prod.fit_predict(df_hist_normal)

cl_name = 'hist_labels'
df_normal[cl_name] = hist_labels
df_hist_normal[cl_name] = hist_labels

cl_names.add(cl_name)

In [ ]:
df_hist_normal.hist_labels.value_counts()

## Testing on K-means and Hierarchical clustering
Based on (1) our previous tests and (2) the context of this problem, the optimal number of clusters is expected to be between 3 and 7.

In [ ]:
def get_ss(df):
    """Computes the sum of squares for all variables given a dataset
    """
    ss = np.sum(df.var() * (df.count() - 1))
    return ss  # return sum of sum of squares of each df variable

def r2(df, labels):
    sst = get_ss(df)
    ssw = np.sum(df.groupby(labels).apply(get_ss))
    return 1 - ssw/sst
    
def get_r2_scores(df, clusterer, min_k=2, max_k=10):
    """
    Loop over different values of k. To be used with sklearn clusterers.
    """
    r2_clust = {}
    for n in range(min_k, max_k):
        clust = clone(clusterer).set_params(n_clusters=n)
        labels = clust.fit_predict(df)
        r2_clust[n] = r2(df, labels)
    return r2_clust


# Set up the clusterers
kmeans = KMeans(
    init='k-means++',
    n_init=20,
    random_state=1
)

hierarchical = AgglomerativeClustering(
    affinity='euclidean'
)

### Finding the optimal clusterer on promotion history variables

#### R2

In [ ]:
# Obtaining the R² scores for each cluster solution on promotion history variables
find_hist_k = False
if find_hist_k:
    r2_scores = {}
    
    print('kmeans')
    r2_scores['kmeans'] = get_r2_scores(df_hist_normal, kmeans)

    if False: 
        for linkage in ['ward']: # 'complete', 'average', 'single',
            print(linkage)
            r2_scores[linkage] = get_r2_scores(
                df_hist_normal, hierarchical.set_params(linkage=linkage)
            )

    pd.DataFrame(r2_scores)

In [ ]:
# Visualizing the R² scores for each cluster solution on demographic variables
if find_hist_k:
    pd.DataFrame(r2_scores).plot.line(figsize=(10,7))

    plt.title("Demographic Variables:\nR² plot for various clustering methods\n", fontsize=21)
    plt.legend(title="Cluster methods", title_fontsize=11)
    plt.xlabel("Number of clusters", fontsize=13)
    plt.ylabel("R² metric", fontsize=13)
    plt.show()

#### Intertia

In [ ]:
inertia = False

if inertia:
    range_clusters = range(1, 11)
    inertia = []
    for n_clus in range_clusters:  # iterate over desired ncluster range
        print(n_clus)
        kmclust = KMeans(n_clusters=n_clus, init='k-means++', n_init=20, random_state=1)
        kmclust.fit(df_normal[cl_feat_hist])
        inertia.append(kmclust.inertia_)  # save the inertia of the given cluster solution

In [ ]:
if inertia:
    plt.plot(inertia)
    plt.show()

#### SOM

In [ ]:
som = False

if som:

    # This som implementation does not have a random seed parameter
    # We're going to set it up ourselves
    np.random.seed(42)

    # Notice that the SOM did not converge - We're under a time constraint for this class
    sm = sompy.SOMFactory().build(
        df[metric_features].values, 
        mapsize=(50, 50), 
        initialization='random',
        neighborhood='gaussian',
        training='batch',
        lattice='hexa',
        component_names=metric_features
    )
    sm.train(n_job=-1, verbose='info', train_rough_len=100, train_finetune_len=100)

In [ ]:
if som:

    # Coordinates of the units in the input space
    sm.get_node_vectors()

In [ ]:
if som:

    # Component planes on the 50x50 grid
    sns.set()
    view2D = View2D(12,12,"", text_size=10)
    view2D.show(sm, col_sz=3, what='codebook')
    plt.subplots_adjust(top=0.90)
    plt.suptitle("Component Planes", fontsize=20)
    plt.show()

In [ ]:
if som:

    # U-matrix of the 50x50 grid
    u = sompy.umatrix.UMatrixView(12, 12, 'umatrix', show_axis=True, text_size=8, show_text=True)

    UMAT = u.show(
        sm, 
        distance2=1, 
        row_normalized=False, 
        show_data=False, 
        contooor=True # Visualize isomorphic curves
    )

## Cluster visualization using t-SNE

In [ ]:
tnse = False

if tnse:
    label_name = 'hist_labels' # 'merged_labels'

    tsne_df = df_normal.sample(frac=.1, axis=0, random_state=1)
    tsne_feat = tsne_df[metric_features]
    tsne_feat

    tsne_c = tsne_df[label_name]
    tsne_c

In [ ]:
if tnse:

    # This is step can be quite time consuming

    two_dim = TSNE(random_state=1, n_jobs=-1).fit_transform(tsne_feat)

In [ ]:
if tnse:

    # t-SNE visualization
    pd.DataFrame(two_dim).plot.scatter(x=0, y=1, c=tsne_c, colormap='tab10', figsize=(15,10))
    plt.show()

In [ ]:
if tnse:

    # t-SNE visualization
    pd.DataFrame(two_dim).plot.scatter(x=0, y=1, c=tsne_c, colormap='tab10', figsize=(15,10))
    plt.show()

# profiling

## feat importance 

In [ ]:
# https://github.com/scikit-learn-contrib/boruta_py
# https://towardsdatascience.com/feature-selection-with-borutapy-f0ea84c9366


# load X and y
# NOTE BorutaPy accepts numpy arrays only, hence the .values attribute
# X = pd.read_csv('examples/test_X.csv', index_col=0).values
# y = pd.read_csv('examples/test_y.csv', header=None, index_col=0).values
# y = y.ravel()

def feat_imp_boruta(X_df,y_s, boruta:True): 
    if boruta: 
        X_features = X_df.columns.to_list()
        X = X_df.values
        y = y_s.values

        print('X.shape:', X.shape)

        # define random forest classifier, with utilising all cores and
        # sampling in proportion to y labels
        rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

        # define Boruta feature selection method
        feat_selector = BorutaPy(rf, n_estimators='auto', verbose=0, random_state=1)

        # find all relevant features - 5 features should be selected
        feat_selector.fit(X, y)

        # check selected features - first 5 features are selected
        print(feat_selector.support_)
        # check ranking of features
        print(feat_selector.ranking_)

        # call transform() on X to filter it down to selected features
        X = feat_selector.transform(X)


        # impt
        X_metadata = pd.DataFrame({
        'features': (X_features),
        'support_b': (feat_selector.support_) 
        #,'ranking': (feat_selector.ranking_)
        })


        print(X_metadata.support_b.sum())

        imp_b = X_metadata.set_index('features')

    else: 
        imp_b = pd.DataFrame({'support_b':[np.nan]}, index=df_normal.drop(columns='hist_labels').columns)
        imp_b.index.set_names('features', inplace=True)
        imp_b
    
    return imp_b

def feat_imp_std(df, cl_name, th):
    
    imp = df.groupby(cl_name).mean().std().sort_values(ascending=False)
    imp = imp.to_frame(name='std')
    imp['std_cumsum'] = imp.cumsum() / imp.cumsum().max()
    imp

    sns.lineplot(data=imp, x=imp.index, y='std_cumsum')
    plt.xticks(rotation=90)
    plt.show()

    imp['support_std'] = [True if v <=th else False for v in imp.std_cumsum]
    imp
    imp.index.set_names('features', inplace=True)
    return imp 


def feat_imp(df, cl_name, cl_names_all, boruta): 
    cl_drop = [x for x in ll if x != cl_names_all]
    
    df = df.drop(cl_drop, errors='ignore')

    
    # boruta
    imp_b = feat_imp_boruta(
        X_df=df.drop(columns=cl_name, errors='ignore'),
        y_s=df[cl_name], 
        boruta = boruta
    )


    # std
    imp_std = feat_imp_std(
        df = df,
        cl_name = cl_name, 
        th = .8
    )


    # combine
    imp_comb = pd.concat([imp_b, imp_std], 1)
    
    # overall support
    cols_with_support = ['support_b', 'support_std']
    imp_comb['support'] = imp_comb[cols_with_support].apply(lambda x: np.sum(x), axis=1)
    
    return imp_comb


In [ ]:
# history

imp_comb_hist = feat_imp(df=df_hist_normal,cl_name='hist_labels',cl_names_all=cl_names, boruta=True)
imp_comb_hist



## mean

## leverage

In [ ]:

def calc_leverage_values(x): 
    d = {}
    d['total_donations'] = x['donations_total'].sum()
    d['size'] = x.size
    return pd.Series(d)

def build_leverage_df(df, cl_name): 
    leverage_df = df_normal.groupby(cl_name).apply(calc_leverage_values)
    leverage_df['total_donations_rel'] = leverage_df['total_donations'] / leverage_df['total_donations'].sum()
    leverage_df['size_rel'] = leverage_df['size'] / leverage_df['size'].sum()

    leverage_df['leverage'] = round(leverage_df['total_donations_rel'] / leverage_df['size_rel'], 2)
    leverage_df.reset_index(inplace=True)
    return leverage_df

def build_leverage_plotdata(leverage_df, cl_name): 

    # bars
    lev_plotdata = leverage_df[[cl_name, 'total_donations_rel', 'size_rel']].melt(id_vars=cl_name)
    lev_plotdata[cl_name] = lev_plotdata[cl_name].astype(str)

    # text annotations
    max_val_per_cl = leverage_df[['total_donations_rel', 'size_rel']].max(1)
    dist = max_val_per_cl.max() * 0.05
    y_coord = max_val_per_cl + dist
    y_coord_df = y_coord.reset_index(name='y_coord')
    y_coord_df.rename(columns={'index':cl_name}, inplace=True)

    lev_text = leverage_df[[cl_name, 'leverage']]
    #lev_text.loc[:,'y_coord'] = y_coord
    lev_text = lev_text.merge(y_coord_df, on=cl_name)
    lev_text

    return lev_plotdata, lev_text

def plot_leverage(lev_plotdata, lev_text, cl_name, plot_name): 

    g = sns.barplot(data=lev_plotdata, x = cl_name, y='value', hue='variable')

    for _, (x,s,y) in lev_text.iterrows():
        plt.text(x=x,y=y, s=s, horizontalalignment='center')

    g.set_ylim(top=g.get_ylim()[1]*1.1)

    plt.tight_layout()
    #print(g.get_ylim())
    plt.savefig(os.path.join(explorations_data_path, f'leverage_{plot_name}.jpeg'), dpi=200)
    plt.show()

def build_plot_leverage(df, cl_name, plot_name): 
    leverage_df = build_leverage_df(df, cl_name)
    lev_plotdata, lev_text = build_leverage_plotdata(leverage_df, cl_name)
    plot_leverage(lev_plotdata, lev_text, cl_name, plot_name)
    return leverage_df


leverage_df = build_plot_leverage(
    df=df_normal, 
    cl_name='hist_labels', 
    plot_name='hist_test'
)

#leverage_df


## mean

In [ ]:
def cluster_profiles(df, label_columns, figsize, compar_titles=None):
    """
    Pass df with labels columns of one or multiple clustering labels. 
    Then specify this label columns to perform the cluster profile according to them.
    """
    if compar_titles == None:
        compar_titles = [""]*len(label_columns)
        
    sns.set()
    fig, axes = plt.subplots(nrows=len(label_columns), ncols=2, figsize=figsize, squeeze=False)
    for ax, label, titl in zip(axes, label_columns, compar_titles):
        # Filtering df
        drop_cols = [i for i in label_columns if i!=label]
        dfax = df.drop(drop_cols, axis=1)
        
        # Getting the cluster centroids and counts
        centroids = dfax.groupby(by=label, as_index=False).mean()
        counts = dfax.groupby(by=label, as_index=False).count().iloc[:,[0,1]]
        counts.columns = [label, "counts"]
        
        # Setting Data
        pd.plotting.parallel_coordinates(centroids, label, color=sns.color_palette(), ax=ax[0])
        sns.barplot(x=label, y="counts", data=counts, ax=ax[1])

        #Setting Layout
        handles, _ = ax[0].get_legend_handles_labels()
        cluster_labels = ["Cluster {}".format(i) for i in range(len(handles))]
        ax[0].annotate(text=titl, xy=(0.95,1.1), xycoords='axes fraction', fontsize=13, fontweight = 'heavy') 
        ax[0].legend(handles, cluster_labels) # Adaptable to number of clusters
        ax[0].axhline(color="black", linestyle="--")
        ax[0].set_title("Cluster Means - {} Clusters".format(len(handles)), fontsize=13)
        ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=30)
        ax[1].set_xticklabels(cluster_labels)
        ax[1].set_xlabel("")
        ax[1].set_ylabel("Absolute Frequency")
        ax[1].set_title("Cluster Sizes - {} Clusters".format(len(handles)), fontsize=13)
    
    #plt.subplots_adjust(hspace=0.4, top=0.90)
    plt.tight_layout()
    plt.title = ''
#    plt.suptitle("Cluster Simple Profilling", fontsize=23)

    filename = ''.join(label_columns)
    plt.savefig(os.path.join(explorations_data_path, f'profile_mean_{filename}.jpeg'), dpi=200)

    plt.show()

In [ ]:
# history
feat_supp_hist = imp_comb_hist.loc[imp_comb_hist.support > 1,].index.to_list()
feat_supp_hist
cl_name_hist = 'hist_labels'
df_hist_normal[feat_supp_hist + [cl_name_hist]].groupby(cl_name_hist).mean().T

my_dpi = 200
figsize=(
    # 15,13
    1800/my_dpi, 1000/my_dpi
)
    
cluster_profiles(
    df = df_hist_normal[feat_supp_hist + [cl_name_hist]], 
    label_columns = [cl_name_hist], 
    figsize = figsize, #None,#(28, 13), 
    compar_titles = ["History clustering"]
)

In [ ]:

# Profilling each cluster (product, behavior, merged)
cluster_profiles(
    df = df[metric_features.to_list() + ['product_labels', 'behavior_labels', 'merged_labels']], 
    label_columns = ['product_labels', 'behavior_labels', 'merged_labels'], 
    figsize = (28, 13), 
    compar_titles = ["Product clustering", "Behavior clustering", "Merged clusters"]
)